# CNN对图片进行分类

In [24]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder ,DatasetFolder
from torch.utils.data import ConcatDataset, DataLoader, Subset,Dataset
import torch
from tqdm.auto import tqdm
import torchvision

## Dataset, Data Loader, and Transforms

In [ ]:
trainTfm =transforms.Compose([transforms.Resize((128,128)),

transforms.ToTensor(),
#transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
testTfm =transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])

In [ ]:
#? 更大的批量通常会产生更稳定的梯度
batch_size = 128
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trainSet = ImageFolder("../data/food-11/training/labeled/",transform=trainTfm)
unlabelSet = ImageFolder("../data/food-11/training/unlabeled/",transform=trainTfm)
validSet = ImageFolder("../data/food-11/validation/",transform=testTfm)
testSet = ImageFolder("../data/food-11/testing/",transform=testTfm)

pm = True if device=="cuda:0" else False   #如果是GPU 把图片加入到CUDA中的固定内存
trainLoader = DataLoader(trainSet,batch_size=batch_size,shuffle=True,num_workers=8, pin_memory=pm)
validLoader = DataLoader(validSet,batch_size=batch_size,shuffle=True,num_workers=8, pin_memory=pm)
testLoader = DataLoader(testSet,batch_size=batch_size,num_workers=8, pin_memory=pm)

## Modle

In [ ]:
class CNNModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.cnnLayers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, 3, 1, 1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2, 0),

            torch.nn.Conv2d(64, 128, 3, 1, 1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2, 0),

            torch.nn.Conv2d(128, 256, 3, 1, 1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(4, 4, 0),
        )
        self.fcLayers = torch.nn.Sequential(
            torch.nn.Linear(256 * 8 * 8, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 11)
        )

    def forward(self, x):      
        x = self.cnnLayers(x)
        x = x.flatten(1)
        x = self.fcLayers(x)
        return x

## Training

In [ ]:
class customSubset(Dataset):
    #自定义DataSet,用于半监督学习未标签的图片，在训练后生成伪标签后组成数据集
    def __init__(self, dataset, indices, labels):
        self.dataset = torch.utils.data.Subset(dataset, indices)
        self.targets = labels
    def __getitem__(self, idx):
        image = self.dataset[idx][0]
        target = self.targets[idx]
        return (image, target)

    def __len__(self):
        return len(self.targets)


def getPseudoLabels(dataset, model, threshold=0.65):
    # 此函数使用给定的模型生成数据集的伪标签
    # 它返回一个DatasetFolder实例，其中包含预测可信度超过给定阈值的图像 
   
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False) 
    model.eval()    
    softmax = torch.nn.Softmax(dim=-1)
    
    for batch in tqdm(data_loader):
        img, _ = batch      
        with torch.no_grad():
            logits = model(img.to(device))
        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)
        # ---------- TODO ----------
        # Filter the data and construct a new dataset.
        

    # # Turn off the eval mode.
    model.train()
    return dataset

In [ ]:
model = CNNModel().to(device)
opt = torch.optim.Adam(model.parameters(),lr=0.0003,weight_decay=1e-5)
criterion = torch.nn.CrossEntropyLoss() 
epochs = 80
doSemi = False
bestAcc = 0
trainLossList ,trainAccList,validLossList,validAccList = [],[],[],[]
for epoch in  tqdm(range(epochs)):    
    if doSemi:
        pseudoSet = getPseudoLabels(unlabelSet,model)
        concatSet = ConcatDataset([trainSet,pseudoSet])
        trainLoader = DataLoader(concatSet,batch_size=batch_size,shuffle=True,num_workers=8,pin_memory=pm)

    model.train()
    trainLoss, trainAcc= [],[]
    for imgs,labels in trainLoader:
        ret = model(imgs.to(device))
        loss = criterion(ret,labels.to(device))
        opt.zero_grad()
        loss.backward()
        #?梯度减切Gradient Clip设置一个梯度减切的阈值，如果在更新梯度的时候，
        #? 梯度超过这个阈值，则会将其限制在这个范围之内，防止梯度爆炸。
        gradNorm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        opt.step()
        acc = (torch.argmax(ret,dim=1)==labels.to(device)).float().mean()
        trainLoss.append(loss)
        trainAcc.append(acc)

    trainLoss,trainAcc = sum(trainLoss)/len(trainLoss),sum(trainAcc)/len(trainAcc)  
    trainLossList.append(trainLoss)
    trainAccList.append(trainAcc)

    model.eval()
    validLoss ,validAcc= [],[]
    for imgs,labels in validLoader:
        with torch.no_grad():
            ret = model(imgs.to(device))
            loss = criterion(ret,labels.to(device))
        acc = (torch.argmax(ret,dim=1)==labels.to(device)).float().mean()
        validLoss.append(loss)
        validAcc.append(acc)

    validLoss ,validAcc= sum(validLoss)/len(validLoss),sum(validAcc)/len(validAcc)     
    validLossList.append(validLoss)
    validAccList.append(validAcc)

    if epoch%5==0:
        print(f"Train--epoch:{epoch}/{epochs}--loss={trainLoss:.5f}--acc={trainAcc:.5f}")
        print(f"*********Valid--loss={validLoss:.5f}--acc={validAcc:.5f}")

    if validAcc > bestAcc:
        bestAcc = validAcc
        torch.save(model.state_dict(),"./bestAcc.ckpt")
        







In [70]:
vgg = torchvision.models.vgg11_bn()
vgg.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU(inplace=True)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [72]:
aa = vgg.classifier
aa._modules['0'] =torch.nn.Linear(in_features=512*4*4, out_features=4096, bias=True)
aa._modules['6'] =torch.nn.Linear(in_features=4096, out_features=11, bias=True)

In [73]:
aa

Sequential(
  (0): Linear(in_features=8192, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=11, bias=True)
)